In [1]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport


# Read json files

In [2]:
alert_df=pd.read_json("../data/alerts_cleaned.json")

In [3]:
corazon_df= pd.read_json("../data/notification-labels-to-alert-surrogate-ids.json")

In [4]:
notification_df=pd.read_json("../data/notifications.json")

# Data Frame Cleaning

## Text managing 

In [5]:
#!pip install -U spacy
#!pip install -U pip setuptools wheel
#!python -m spacy download es_core_news_sm
#!pip install spacymoji
!python -m spacy download es_core_news_lg
import spacy
from spacymoji import Emoji
import string
import re
import es_core_news_lg
nlp = es_core_news_lg.load()

     |████████████████████████████████| 568.4 MB 6.1 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [6]:
import emoji
def get_emoji(x):

    doc = nlp(x)
    l=[]
    for i in doc:
        i=i.orth_
        if i in emoji.UNICODE_EMOJI : 
            l.append(i)
    return l 

In [7]:
def rid_emoji(x,y):
    doc = nlp(x)
    l=[]
    for i in doc:
        i=i.orth_
        if i in y:
            pass
        else: 
            l.append(i)
    return ' '.join(l)

In [8]:
def slang_sep(x):
    if '?' in x:
        if x.count("?")>1:
            x=re.compile(r'[?|!]').split(x,x.count('?'))
            return ["".join(x[0]+ x[1]),x[2]]
        else:return re.compile(r'[?|!]').split(x,1)
    elif '!' in x:return re.compile(r'[?|!]').split(x,1) 
    else:  return ['',x]


In [9]:
def label_text (x,y):
    c=0
    for token in  nlp(x):
        if token.tag_== y: c=c+1
        else: pass  
    return c  

In [10]:
def text_rev (x):
    for token in  nlp(x):
        print([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_,
                 token.is_alpha, token.is_stop])

In [11]:
def words_count (x):
   return len(nlp(x))
    

In [12]:
alert_df['description']     = alert_df['description'].astype('string')
alert_df['lenght']          = alert_df['description'].str.len()
alert_df["emoji"]           = alert_df["description"].apply(lambda x: get_emoji(x))
alert_df["emoji_size"]      = alert_df["emoji"].str.len()
alert_df["txt_description"] = alert_df.apply(lambda x: rid_emoji(x.description,x.emoji),axis=1)

In [13]:
alert_df["slang"] = alert_df["txt_description"].apply(lambda x: slang_sep(x))
alert_df[['slang','information']] = pd.DataFrame(alert_df["slang"].tolist(), index= alert_df.index)
alert_df.drop(["txt_description"],axis=1, inplace= True)

In [14]:
alert_df["slang_char"] = alert_df["slang"].str.len()
alert_df["slang_verb"] = alert_df["slang"].apply(lambda x:  label_text (x,'VERB'  ))
alert_df["slang_pron"] = alert_df["slang"].apply(lambda x:  label_text (x,'PROPN' ))
alert_df["slang_adp" ] = alert_df["slang"].apply(lambda x:  label_text (x,'ADP'   ))
alert_df["slang_noun"] = alert_df["slang"].apply(lambda x:  label_text (x,'NOUN'  ))
alert_df["slang_num" ] = alert_df["slang"].apply(lambda x:  label_text (x,'NUM'   ))
alert_df["slang_punt"] = alert_df["slang"].apply(lambda x:  label_text (x,'PUNCT' ))
alert_df["slang_det" ] = alert_df["slang"].apply(lambda x:  label_text (x,'DET'   ))

In [15]:
alert_df["info_char" ]  = alert_df["information"].str.len()
alert_df["info_words"]  = alert_df["information"].apply(lambda x:  words_count (x))
alert_df["info_verb" ]  = alert_df["information"].apply(lambda x:  label_text  (x,'VERB'  ))
alert_df["info_pron" ]  = alert_df["information"].apply(lambda x:  label_text  (x,'PROPN' ))
alert_df["info_adp"  ]  = alert_df["information"].apply(lambda x:  label_text  (x,'ADP'   ))
alert_df["info_noun" ]  = alert_df["information"].apply(lambda x:  label_text  (x,'NOUN'  ))
alert_df["info_num"  ]  = alert_df["information"].apply(lambda x:  label_text  (x,'NUM'   ))
alert_df["info_punt" ]  = alert_df["information"].apply(lambda x:  label_text  (x,'PUNCT' ))
alert_df["info_det"  ]  = alert_df["information"].apply(lambda x:  label_text  (x,'DET'   ))

In [17]:
alert_df

,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,...,slang_det,info_char,info_words,info_verb,info_pron,info_adp,info_noun,info_num,info_punt,info_det
0,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22,2021-02-24 03:58:39,2021-02-24 04:21:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,GtvOEQAFZ0GtU6u4AXwvPg,...,0,101,16,2,3,4,4,1,1,0
1,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22,2021-02-24 03:58:39,2021-02-24 04:21:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,JfA8Bw8Zp024Kqu4AXiSpQ,...,0,101,16,2,3,4,4,1,1,0
2,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55,2021-03-31 22:54:19,2021-04-01 00:01:01,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,GtvOEQAFZ0GtU6u4AXwvPg,...,0,114,19,1,4,4,5,0,1,1
3,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55,2021-03-31 22:54:19,2021-04-01 00:01:01,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,NuuRQ2I1Q0a50Kv-AVKlLA,...,0,114,19,1,4,4,5,0,1,1
4,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55,2021-03-31 22:54:19,2021-04-01 00:01:01,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,zCy9zG00HEqGeKu4AWZYNQ,...,0,114,19,1,4,4,5,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13537,-MZZdjyfbnztCccgqoa1,sEfkzvmp6xG9Us96D1NGhQ,2021-04-30 21:19:54,2021-04-30 21:32:24,2021-04-30 22:14:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Oh que la! Afectación vial por semáforos de...,false,zCy9zG00HEqGeKu4AWZYNQ,...,1,101,17,0,6,4,3,0,1,0
13538,-MZor-DYgDMER9B2NtGD,QHbP5XKs6xGbFYutnY_XdQ,2021-05-04 00:51:44,2021-05-04 00:52:21,2021-05-04 01:45:01,BrokenTrafficLight,TrafficImpact,🚦 ¡Nos lleva el chanfle! Afectación vial por s...,false,zCy9zG00HEqGeKu4AWZYNQ,...,1,98,16,0,4,5,3,0,1,0
13539,-MWizRKZ_YIPcMWzaqMP,QNJvrEGO6xGgXg3PRUP4fg,2021-03-26 14:43:48,2021-03-26 14:44:15,2021-03-26 16:01:01,BrokenTrafficLight,TrafficImpact,⏳ ¡No te esponjes! Hay retrasos por semáforos ...,false,zCy9zG00HEqGeKu4AWZYNQ,...,0,110,21,0,4,5,5,0,1,2
13540,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46,2021-01-27 17:51:51,2021-01-27 19:01:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,...,0,82,13,0,4,2,2,0,1,0


## Sentiment analysis

In [28]:
#!pip install transformers

  Using cached transformers-4.9.0-py3-none-any.whl (2.6 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-macosx_10_11_x86_64.whl (2.2 MB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)


In [30]:
#!pip3 install torch torchvision torchaudio

     |████████████████████████████████| 127.9 MB 3.5 MB/s 
     |████████████████████████████████| 13.9 MB 1.0 MB/s 
     |████████████████████████████████| 1.5 MB 3.8 MB/s 


In [14]:
import transformers
import torch
from transformers import pipeline

nlp_ner = pipeline(
    "ner",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    tokenizer=(
        'mrm8488/bert-spanish-cased-finetuned-ner',  
        {"use_fast": False}
))
#text = 'Mis amigos están pensando viajar a Londres este verano'

#nlp_ner(text)

Downloading: 100%|██████████| 439M/439M [02:17<00:00, 3.20MB/s]
Downloading: 100%|██████████| 136/136 [00:00<00:00, 25.9kB/s]
Downloading: 100%|██████████| 242k/242k [00:00<00:00, 1.20MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 10.8kB/s]


In [16]:
alert_df["information"][0]

'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre .'

In [15]:
nlp_ner(alert_df["information"][0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOC',
  'score': 0.999429,
  'index': 10,
  'word': 'Meso',
  'start': None,
  'end': None},
 {'entity': 'B-LOC',
  'score': 0.99264264,
  'index': 11,
  'word': '##nes',
  'start': None,
  'end': None},
 {'entity': 'B-LOC',
  'score': 0.99626726,
  'index': 13,
  'word': 'Av',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.99655366,
  'index': 14,
  'word': '.',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.99839115,
  'index': 15,
  'word': '20',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.99911416,
  'index': 16,
  'word': 'de',
  'start': None,
  'end': None},
 {'entity': 'I-LOC',
  'score': 0.9988262,
  'index': 17,
  'word': 'Noviembre',
  'start': None,
  'end': None}]

In [21]:
alert_df.tail(30)


,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates_latitude,area_of_effect_coordinates_longitude,lenght,emoji,emoji_size,txt_description,slang,slang_size,information,information_size
13512,-MdZcijmD1TYPV79cFWv,IN2hL83a6xGRhiXG-lTJaQ,2021-07-02 00:33:56,2021-07-02 00:35:01,2021-07-02 01:44:01,BrokenTrafficLight,Delays,🚦 ¡Chanclas! Retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,-99.190089,19.346380,84,[🚦],1,¡ Chanclas ! Retrasos por semáforos descompues...,¡ Chanclas,2,Retrasos por semáforos descompuestos en Av. d...,73
13513,-McESSE8vMINToR_NWy1,kBsi_s3N6xGOzAnrpNUb0A,2021-06-15 11:36:58,2021-06-15 11:37:05,2021-06-15 12:32:01,BrokenTrafficLight,LaneReduction,⏳🚦 ¡Úchalas! Retrasos por semáforos descompues...,false,GtvOEQAFZ0GtU6u4AXwvPg,-99.100600,19.385777,107,"[⏳, 🚦]",2,¡ Úchalas ! Retrasos por semáforos descompuest...,¡ Úchalas,2,Retrasos por semáforos descompuestos en Canal...,96
13514,-McESSE8vMINToR_NWy1,kBsi_s3N6xGOzAnrpNUb0A,2021-06-15 11:36:58,2021-06-15 11:37:05,2021-06-15 12:32:01,BrokenTrafficLight,LaneReduction,⏳🚦 ¡Úchalas! Retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,-99.100600,19.385777,107,"[⏳, 🚦]",2,¡ Úchalas ! Retrasos por semáforos descompuest...,¡ Úchalas,2,Retrasos por semáforos descompuestos en Canal...,96
13515,-MYR1vI2STFTeww4Dfvh,QGjGnOWe6xGAF0VdZdpUyw,2021-04-16 18:57:37,2021-04-16 19:32:49,2021-04-16 20:01:02,BrokenTrafficLight,Delays,⏳ ¡Ah caray! Hay retrasos por servicios de eme...,false,GtvOEQAFZ0GtU6u4AXwvPg,-99.128356,19.459590,137,[⏳],1,¡ Ah caray ! Hay retrasos por servicios de eme...,¡ Ah caray,2,Hay retrasos por servicios de emergencia en C...,128
13516,-MYR1vI2STFTeww4Dfvh,QGjGnOWe6xGAF0VdZdpUyw,2021-04-16 18:57:37,2021-04-16 19:32:49,2021-04-16 20:01:02,BrokenTrafficLight,Delays,⏳ ¡Ah caray! Hay retrasos por servicios de eme...,false,zCy9zG00HEqGeKu4AWZYNQ,-99.128356,19.459590,137,[⏳],1,¡ Ah caray ! Hay retrasos por servicios de eme...,¡ Ah caray,2,Hay retrasos por servicios de emergencia en C...,128
13517,-M_cKywp7tHVqcdhMikg,UEZx_k206xGz_YW87SnlaA,2021-05-14 00:47:43,2021-05-14 00:48:04,2021-05-14 01:45:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Híjole! Afectación vial por semáforos desco...,false,GtvOEQAFZ0GtU6u4AXwvPg,-99.280258,19.360887,98,[🚦],1,¡ Híjole ! Afectación vial por semáforos desco...,¡ Híjole,2,Afectación vial por semáforos descompuestos e...,88
13518,-M_cKywp7tHVqcdhMikg,UEZx_k206xGz_YW87SnlaA,2021-05-14 00:47:43,2021-05-14 00:48:04,2021-05-14 01:45:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Híjole! Afectación vial por semáforos desco...,false,zCy9zG00HEqGeKu4AWZYNQ,-99.280258,19.360887,98,[🚦],1,¡ Híjole ! Afectación vial por semáforos desco...,¡ Híjole,2,Afectación vial por semáforos descompuestos e...,88
13519,-MZrFImL2uzTg2cW3kIy,YOPPd9Cs6xGaMLdLw12PeA,2021-05-04 12:01:32,2021-05-04 12:45:26,2021-05-04 12:55:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Aguas! Afectación vial por semáforos descom...,false,GtvOEQAFZ0GtU6u4AXwvPg,-99.196489,19.511512,111,[🚦],1,¡ Aguas ! Afectación vial por semáforos descom...,¡ Aguas,2,Afectación vial por semáforos descompuestos e...,104
13520,-MZrFImL2uzTg2cW3kIy,YOPPd9Cs6xGaMLdLw12PeA,2021-05-04 12:01:32,2021-05-04 12:45:26,2021-05-04 12:55:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Aguas! Afectación vial por semáforos descom...,false,zCy9zG00HEqGeKu4AWZYNQ,-99.196489,19.511512,111,[🚦],1,¡ Aguas ! Afectación vial por semáforos descom...,¡ Aguas,2,Afectación vial por semáforos descompuestos e...,104
13521,-MZVAWYjvUbZ4U7wVkcT,8DcvHEup6xGi5leOp8mbSA,2021-04-30 00:29:22,2021-04-30 00:29:27,2021-04-30 01:26:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Úchala! Afectación vial por semáforos apaga...,false,GtvOEQAFZ0GtU6u4AXwvPg,-99.077524,19.378870,102,[🚦],1,¡ Úchala ! Afectación vial por semáforos apaga...,¡ Úchala,2,Afectación vial por semáforos apagados en Eje...,93


In [234]:
#print(alert_df['txt_description'].loc[alert_df['document_id']=='-M_wOdT14xRsVVEViOK1']

303    ¿ A esperar bajo la lluvia 2 horas ? ¿ Mil hor...
Name: txt_description, dtype: object

## Merging alert df with agency dummy variables

In [19]:
alert2= alert_df[['surrogate_id','agency']].set_index('surrogate_id')
alert2= pd.get_dummies(alert2)
grouped_alert= alert2.groupby(by='surrogate_id').sum()
alert_df.drop(['agency'], axis=1, inplace=True)
alert_df.drop_duplicates(subset='surrogate_id',inplace=True,ignore_index=True)
alert_df= pd.merge(alert_df,grouped_alert, on='surrogate_id')

## Merging notification df with corazon df

In [20]:
notification_df.drop_duplicates(inplace=True,ignore_index=True)
corazon_df.drop(['day'], axis=1, inplace=True)
corazon_df.drop_duplicates(inplace=True, ignore_index=True)
real_merge= notification_df.merge(corazon_df, how="left", left_on="join_key_value", right_on="notification_label_id")

## Merging all df together

In [7]:
real_merge.shape

(2274889, 8)

In [8]:
#Queried df for proper merging
label_merger= real_merge.query("join_field == 'label'")
alert_id_merger= real_merge.query("join_field == 'alertId'")

In [9]:
clean_df=label_merger.merge(alert_df, how="left",left_on="corazon_surrogate_id", right_on="surrogate_id")

In [10]:
clean_df_3=alert_id_merger.merge(alert_df, how="left",left_on="join_key_value", right_on="document_id")

In [11]:
clean_df_2=clean_df[clean_df['description'].isnull()]
clean_df.dropna(subset=['description'],inplace=True)
clean_df_2=clean_df_2[["event_date","event_timestamp", "event_name","user_id","join_field","join_key_value"]]
clean_df_4=clean_df_2.merge(alert_df, how="left",left_on="join_key_value", right_on="document_id")
clean_df_4.dropna(subset=['created_at'],inplace=True)

#Error! Notification open cant be related to any alerts. Ask

In [13]:
# Finding more problems

In [14]:
clean_df_3["join_key_value"].value_counts()

null                    484
-MddArdZ4i8kL5p7wG0H    466
-MdYiDEUKL6_4QGni-7o    421
-Me5bbxeM43sR8EbuLd2    386
-Mdt2BT-2y05YXyJctt2    371
                       ... 
-Mde9L0oTy1vODzHW294      1
-Mb2OcjgCicoGbENOVIQ      1
-McEmQIbv558V8RxXBIU      1
-Me1JV-RKQX-qj-_7QU8      1
-Ma9-5EDz-3vPh4VZOSb      1
Name: join_key_value, Length: 2289, dtype: int64

In [15]:
clean_df_3.query("event_name == 'notification_opened'")["description"].isnull().value_counts()
# Ask Ellen, 700 are gone

False    2826
True      673
Name: description, dtype: int64

In [16]:
#Another merge

In [17]:
clean_df_3["description"].isnull().value_counts()

False    32180
True      4308
Name: description, dtype: int64

In [18]:
clean_df_3.dropna(subset=['description'],inplace=True)

In [19]:
almost_finished_df= pd.concat([clean_df,clean_df_4], axis=0)
finished_df= pd.concat([almost_finished_df,clean_df_3], axis=0)

In [20]:
finished_df.head()

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,corazon_surrogate_id,notification_label_id,document_id,surrogate_id,...,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ
5,20210529,1622240527928000,notification_receive,ac6d3,label,o3RXFQPA6xGpKOFIXMTiIg,4Np4EQPA6xGpKOFIXMTiIg,o3RXFQPA6xGpKOFIXMTiIg,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,20210529,1622240959952000,notification_dismiss,ac6d3,label,o3RXFQPA6xGpKOFIXMTiIg,4Np4EQPA6xGpKOFIXMTiIg,o3RXFQPA6xGpKOFIXMTiIg,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,20210529,1622248553374000,notification_receive,ac6d3,label,M1Mb7hDA6xGpKOFIXMTiIg,kGA96xDA6xGpKOFIXMTiIg,M1Mb7hDA6xGpKOFIXMTiIg,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,20210529,1622248568285001,notification_dismiss,ac6d3,label,M1Mb7hDA6xGpKOFIXMTiIg,kGA96xDA6xGpKOFIXMTiIg,M1Mb7hDA6xGpKOFIXMTiIg,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10,20210529,1622258011584001,notification_receive,ac6d3,label,E4TxyCvA6xGpKOFIXMTiIg,YKlNxSvA6xGpKOFIXMTiIg,E4TxyCvA6xGpKOFIXMTiIg,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
finished_df.columns

Index(['event_date', 'event_timestamp', 'event_name', 'user_id', 'join_field',
       'join_key_value', 'corazon_surrogate_id', 'notification_label_id',
       'document_id', 'surrogate_id', 'created_at', 'published_at',
       'closed_at', 'cause', 'effect', 'description', 'is_global',
       'area_of_effect_coordinates_latitude',
       'area_of_effect_coordinates_longitude', 'agency_GewRJAw5tUmC4Ku4AX1-SQ',
       'agency_GtvOEQAFZ0GtU6u4AXwvPg', 'agency_HE59N3RXM0q5vKu4AXlQZg',
       'agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ',
       'agency_MgUq5b9mOEunx6u4AXt_BA', 'agency_NuuRQ2I1Q0a50Kv-AVKlLA',
       'agency_V2AIQQKgmUO3VazvAOA-Cw', 'agency_jLjibFoim0iwWau4AWoEdQ',
       'agency_pky7jovXYkaw-awAAMrQ3g', 'agency_zCy9zG00HEqGeKu4AWZYNQ'],
      dtype='object')

## Dropping irrelevant columns

In [22]:
export_df=finished_df.drop(['event_date','join_field','join_key_value','is_global','corazon_surrogate_id','notification_label_id'], axis=1)

## Fixing the event_name column:

In [23]:
export_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [24]:
export_df["event_name"].replace("notification_open","notification_opened", inplace=True)

## Exporting the df to a json file, so it is easier to deal without the cleaning part.

In [25]:
export_df.reset_index(inplace=True, drop=True)

In [26]:
export_df.to_json("../data/cleaned_data.json",orient='columns')

In [27]:
!pip3 install pyarrow
import pyarrow.feather as feather
feather.write_feather(export_df, "../data/cleaned_data.feather")

In [28]:
export_df = feather.read_feather("../data/cleaned_data.feather")